<h1>Índice<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparando-Jupyter" data-toc-modified-id="Preparando-Jupyter-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparando Jupyter</a></span></li><li><span><a href="#Aproximando-o-Gasto-Calórico-Diário" data-toc-modified-id="Aproximando-o-Gasto-Calórico-Diário-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Aproximando o Gasto Calórico Diário</a></span><ul class="toc-item"><li><span><a href="#Exemplo" data-toc-modified-id="Exemplo-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Exemplo</a></span></li></ul></li><li><span><a href="#Preparando-Dados" data-toc-modified-id="Preparando-Dados-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preparando Dados</a></span><ul class="toc-item"><li><span><a href="#Distribuição-de-Calorias" data-toc-modified-id="Distribuição-de-Calorias-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Distribuição de Calorias</a></span></li></ul></li><li><span><a href="#Modelo-Matemático" data-toc-modified-id="Modelo-Matemático-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Modelo Matemático</a></span></li><li><span><a href="#Mais-detalhes-ao-modelo" data-toc-modified-id="Mais-detalhes-ao-modelo-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Mais detalhes ao modelo</a></span></li><li><span><a href="#Verificando-o-Gasto-Calórico-Basal-e-Total" data-toc-modified-id="Verificando-o-Gasto-Calórico-Basal-e-Total-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Verificando o Gasto Calórico Basal e Total</a></span></li><li><span><a href="#Definindo-os-Alimentos" data-toc-modified-id="Definindo-os-Alimentos-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Definindo os Alimentos</a></span></li></ul></div>

# Preparando Jupyter
***

In [1]:
import XLSX
using JuMP, Juniper, Ipopt
using DataFrames

In [2]:
if ~occursin("Dieta", pwd())
    try
        cd("Desktop/Dieta")
    catch
        nothing
    end
end

# Aproximando o Gasto Calórico Diário
***

Gasto Basal por Harris e Benedict:

$ M(kg, cm, idade) = 13,8.kg + 5.cm - 6,8.idade + 66 $\
$ F(kg, cm, idade) = 9,6.kg + 1,9.cm - 4,7.idade + 665 $

In [15]:
#Utilizar com ativos fisicamente, objetivo: aumento de peso e massa magra.
#SUPERESTIMA O BASAL
function basal_Harris_Benedict(peso_kg, altura_cm, idade, sexo = 0)
    if sexo == 0 #homem
        return 66 + (13.8*peso_kg) + (5*altura_cm) - (6.8*idade)
    else #mulher
        return 665 + (9.6*peso_kg) + (1.9*altura_cm) - (4.7*idade)
    end
end

basal_Harris_Benedict (generic function with 2 methods)

Gasto Basal por FAO e OMS:

| Idade   | Masculino           | Feminino            |
|---------|---------------------|---------------------|
| 10 a 18 | (17,686.kg) + 658,2 | (13,384.kg) + 692,6 |
| 18 a 30 | (15,057.kg) + 692,2 | (14,818.kg) + 486,6 |
| 30 a 60 | (11,472.kg) + 873,1 | (8,126.kg) + 845,6  |
| >60     | (11,711.kg) + 587,7 | (9,082.kg) + 658,5  |

In [16]:
function basal_FAO_OMS(peso_kg, idade, sexo = 0)
    if sexo == 0 #homem
        if 10 ≤ idade < 18
            return (17.686*peso_kg) + 658.2
        elseif  18 ≤ idade < 30
            return (15.057*peso_kg) + 692.2
        elseif 30 ≤ idade < 60
            return (11.472*peso_kg) + 873.1
        else
            return (11.711*peso_kg) + 587.7
        end
    else #mulher
        if 10 ≤ idade < 18
            return (13.384*peso_kg) + 692.6
        elseif  18 ≤ idade < 30
            return (14.818*peso_kg) + 486.6
        elseif 30 ≤ idade < 60
            return (8.126*peso_kg) + 845.6
        else
            return 	(9.082*peso_kg) + 658.5
        end
    end
end

basal_FAO_OMS (generic function with 2 methods)

Gasto Basal por Mifflin e St. Jeor:

$ M(kg, cm, idade) = 10.kg + 6,25.cm - 5.idade + 5 $\
$ F(kg, cm, idade) = 10.kg + 6,25.cm - 5.idade - 161 $

In [17]:
#objetivo de emagrecimento.
#MAIS PRECISA QUE Harris_Benedict
function basal_Mifflin_St_Jeor(peso_kg, altura_cm, idade, sexo = 0)
    if sexo == 0 #homem
        return (10*peso_kg) + (6.25*altura_cm) - (5*idade) + 5
    else #mulher
        return (10*peso_kg) + (6.25*altura_cm) - (5*idade) - 161
    end
end

basal_Mifflin_St_Jeor (generic function with 2 methods)

Gasto Basal por Cunningham e Tinsley:

$ T1(Mm) = 22.(Mm) + 500 $\
$ T2(Mm) = 25,9.(Mm) + 284 $

$ T3(kg) = 24,8.kg + 284 $

**Mm** = Massa magra

In [18]:
#individuos ativos fisicamente e que possuem alto volume muscular e baixo percentual de gordura.
function basal_Cunningham_Tinsley(peso_kg, massa_magra; tipo = 1)
    if tipo == 1 #Cunningham (MLG)
        return (22*massa_magra) + 500
    elseif tipo == 2 #Tinsley (MLG)
        return (25.9*massa_magra) + 284
    else #Tinsley (P)
        return (24.8*peso_kg) + 10
    end
end

basal_Cunningham_Tinsley (generic function with 1 method)

## Exemplo

In [19]:
function atividade(atividades, basa)
    gasto_total = 1.
    if atividades == "Sedentario"
        gasto_total = 1.2*basa #1 a 1.39
    elseif atividades == "Pouco ativo"
        gasto_total = 1.45*basa #1.4 a 1.59
    elseif atividades == "Ativo"
        gasto_total = 1.75*basa #1.6 a 1.89
    else #Muito Ativo
        gasto_total = 2.2*basa #1.9 a 2.5
    end
    return gasto_total
end

function basal(kg, altura, age, massa_magra ;pref = 0)
    set_basal = [basal_Harris_Benedict(kg, altura, age),
            basal_FAO_OMS(kg, age),
            basal_Mifflin_St_Jeor(kg, altura, age),
            basal_Cunningham_Tinsley(kg, massa_magra, tipo = 1),
            basal_Cunningham_Tinsley(kg, massa_magra, tipo = 2),
            basal_Cunningham_Tinsley(kg, massa_magra, tipo = 3)]
    nomes = ["Harris e Benedict", "FAO e OMS", "Mifflin e St. Jeor", "Cunningham", "Tinsley Mm", "Tinsley peso", "Média"]
    med = sum(set_basal)/6
    if pref == 0
        return med
    elseif pref == 1
        return (set_basal[1] + set_basal[3])/2
    end
    return DataFrame(Nome = nomes, Basal = append!(set_basal, [med]))
end

basal (generic function with 1 method)

In [20]:
#Exemplo Masculino
kg_m = 64
age_m = 21
altura_m = 162
massa_magra_m = kg_m*(1 - (22.15 + 6)/100) #22.15% de gordura, 6% de óssos e etc.
atividades_m = "Ativo"
""

#Exemplo Feminino
kg_f = 54
age_f = 22
altura_f = 162
massa_magra_f = kg_f*(1 - 0.14 - 0.06) #14% de gordura, 6% de óssos e etc.
atividades_f = "Ativo"
""

""

In [21]:
#Masculino
basal(kg_m, altura_m, age_m, massa_magra_m, pref = 3)

,Nome,Basal
,String,Float64
1,Harris e Benedict,1616.4
2,FAO e OMS,1655.85
3,Mifflin e St. Jeor,1552.5
4,Cunningham,1511.65
5,Tinsley Mm,1474.99
6,Tinsley peso,1597.2
7,Média,1568.1


In [22]:
#Feminino
basal(kg_f, altura_f, age_f, massa_magra_f, pref = 3)

,Nome,Basal
,String,Float64
1,Harris e Benedict,1471.6
2,FAO e OMS,1505.28
3,Mifflin e St. Jeor,1447.5
4,Cunningham,1450.4
5,Tinsley Mm,1402.88
6,Tinsley peso,1349.2
7,Média,1437.81


In [23]:
basal_m = basal(kg_m, altura_m, age_m, massa_magra_m, pref = 1)
basal_f = basal(kg_f, altura_f, age_f, massa_magra_f, pref = 1)

println("Basal Masculino: ", basal_m)
println("Basal Feminino: ", basal_f)

Basal Masculino: 1584.45
Basal Feminino: 1459.5500000000002


In [24]:
gct_m = atividade(atividades_m, basal_m)
gct_f = atividade(atividades_f, basal_f)

println("Gasto Total Calórico Masculino - ", gct_m)
println("Gasto Total Calórico Feminino - ", gct_f)

Gasto Total Calórico Masculino - 2772.7875
Gasto Total Calórico Feminino - 2554.2125000000005


# Preparando Dados
***

In [3]:
xf = XLSX.readxlsx("tabela.xlsx")
tamanho_final = string(xf["Tabela1"])[40:end-1] #tamanho inteiro [35:end-1]  da tabela, "A1:FX203"
sh = xf[string("Tabela1!A1:",tamanho_final)] #pego a tabela hidr do A3 até o tamanho_final
DataFrame(sh[2:end,1:end], Symbol.(sh[1,1:end]))

,Item,Nome,Preparo,Carboidrato (g)
,Any,Any,Any,Any
1,1,Arroz,"integral, cozido","25,8"
2,2,Arroz,"integral, cru","77,5"
3,3,Arroz,"tipo 1, cozido","28,1"
4,4,Arroz,"tipo 1, cru","78,8"
5,5,Arroz,"tipo 2, cozido","28,2"
6,6,Arroz,"tipo 2, cru","78,9"
7,7,Aveia,"flocos, crua","66,6"
8,8,Biscoito,"doce, maisena","75,2"
9,9,Biscoito,"doce, recheado com chocolate","70,5"


In [30]:
elementos = [3; 179; 222; 410; 456; 488; 567; 524; 260]
#peso = [5 2.5 2.5 5 1.5 2.5 2.5 1.5 1.5]
elementos .+= 1
col = [2; 4; 5; 6; 7]
DataFrame(sh[elementos,[2,3]], Symbol.(sh[1,[2,3]]))

,Nome,Preparo
,Any,Any
1,Arroz,"tipo 1, cozido"
2,Banana,"nanica, crua"
3,Maçã,"Fuji, com casca, crua"
4,Frango,"peito, sem pele, grelhado"
5,Leite,"de vaca, desnatado, pó"
6,Ovo,"de galinha, inteiro, cozido/10minutos"
7,Feijão,"preto, cozido"
8,Maionese,tradicional com ovos
9,Azeite,"de oliva, extra virgem"


In [8]:
println(DataFrame(sh[elementos,col], Symbol.(sh[1,col])))

9×6 DataFrame
 Row │ Nome      Preparo                            Carboidrato (g)  Proteína (g)  Lipídeos (g)  Energia (Kcal) 
     │ Any       Any                                Any              Any           Any           Any            
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Arroz      tipo 1, cozido                    28,1             2,5           0,2           128
   2 │ Banana     nanica, crua                      23,8             1,4           0,1           92
   3 │ Maçã       Fuji, com casca, crua             15,2             0,3           Tr            56
   4 │ Frango     peito, sem pele, grelhado         0,0              32,0          2,5           159
   5 │ Leite      de vaca, desnatado, pó            53,0             34,7          0,9           362
   6 │ Ovo        de galinha, inteiro, cozido/10m…  0,6              13,3          9,5           146
   7 │ Feijão     preto, cozido            

| Num | Nome     | Preparo                  | Carboidrato (g) | Proteína (g) | Lipídeos (g) | Energia (Kcal) |
|-----|----------|--------------------------|:---------------:|:------------:|:------------:|:--------------:|
| 1   | Arroz    | tipo 1, cozido           |       28,1      |      2,5     |      0,2     |       128      |
| 2   | Banana   | nanica, crua             |       23,8      |      1,4     |      0,1     |       92       |
| 3   | Maçã     | Fuji, c\ casca, crua     |       15,2      |      0,3     |      Tr      |       56       |
| 4   | Frango   | peito, s\ pele, grelhado |       0,0       |     32,0     |      2,5     |       159      |
| 5   | Leite    | vaca, desnatado, pó      |       53,0      |     34,7     |      0,9     |       362      |
| 6   | Ovo      | de galinha, cozido       |       0,6       |     13,3     |      9,5     |       146      |
| 7   | Feijão   | preto, cozido            |       14,0      |      4,5     |      0,5     |       77       |
| 8   | Maionese | tradicional com ovos     |       7,9       |      0,6     |     30,5     |       302      |
| 9   | Azeite   | de oliva, extra virgem   |        NA       |      NA      |     100,0    |       884      |

In [25]:
Limite_inf, Limite_sup = ones(length(elementos),1),ones(length(elementos),1)

Limite_inf[:] = [100; 100; 100; 100; 50; 50; 50; 50; 10]/100

Limite_sup[:] = [500; 250; 150; 500; 250; 200; 200; 100; 100]/100

Limite_inf = convert(Array{Any,2}, Limite_inf)
Limite_sup = convert(Array{Any,2}, Limite_sup)

DataFrame([Int64.(Limite_inf*100) sh[elementos,[2]] Int64.(Limite_sup*100)], ["Inf g", "Alimento", "Sup g"])

,Inf g,Alimento,Sup g
,Any,Any,Any
1,100,Arroz,500
2,100,Banana,250
3,100,Maçã,150
4,100,Frango,500
5,50,Leite,250
6,50,Ovo,200
7,50,Feijão,200
8,50,Maionese,100
9,10,Azeite,100


In [26]:
Limite_inf = convert(Array{Float64,2}, Limite_inf)
Limite_sup = convert(Array{Float64,2}, Limite_sup)
""

""

In [32]:
df = sh[elementos,col]
aux1 = replace.(df[1:end, 1:end-1], "Tr" => "0")
aux1 = replace.(aux1, "NA" => "0")
aux = parse.(Float64,replace.(aux1[1:end, 2:end], "," => "."))

name = [:Produto , :Carb, :Prot, :Gord, :Calorias, :Inf, :Sup]

df = DataFrame([df[1:end, 1] aux df[1:end, end] Limite_inf Limite_sup],  name)

,Produto,Carb,Prot,Gord,Calorias,Inf,Sup
,Any,Any,Any,Any,Any,Any,Any
1,Arroz,28.1,2.5,0.2,128,1.0,5.0
2,Banana,23.8,1.4,0.1,92,1.0,2.5
3,Maçã,15.2,0.3,0.0,56,1.0,1.5
4,Frango,0.0,32.0,2.5,159,1.0,5.0
5,Leite,53.0,34.7,0.9,362,0.5,2.5
6,Ovo,0.6,13.3,9.5,146,0.5,2.0
7,Feijão,14.0,4.5,0.5,77,0.5,2.0
8,Maionese,7.9,0.6,30.5,302,0.5,1.0
9,Azeite,0.0,0.0,100.0,884,0.1,1.0


## Distribuição de Calorias
***

In [33]:
function distribuicao(p,g, kg, gct;def = 0.2, pref = "gramas")
    dist = [p g].*[4 9]

    prot_cal, gord_cal = kg*dist
    carb_cal = gct - prot_cal - gord_cal
    carb_cal = carb_cal*(1 - def)

    carb_g = carb_cal/4
    prot_g = prot_cal/4
    gord_g = gord_cal/9
    if pref == "gramas"
        return carb_g, prot_g, gord_g
    end
    return carb_cal, prot_cal, gord_cal
end

distribuicao (generic function with 1 method)

In [34]:
prot, gord = 2.0, 1.0

carb_g, prot_g, gord_g = distribuicao(prot, gord, kg_m, gct_m)
carb_cal, prot_cal, gord_cal = distribuicao(prot, gord, kg_m, gct_m, pref = "calorias")

DataFrame([["Carboidratos"; "Proteínas"; "Gorduras"], [carb_g; prot_g; gord_g], [carb_cal; prot_cal; gord_cal]], ["Macros", "Gramas", "Calorias"])

,Macros,Gramas,Calorias
,String,Float64,Float64
1,Carboidratos,336.957,1347.83
2,Proteínas,128.0,512.0
3,Gorduras,64.0,576.0


In [35]:
alimentos = length(elementos)
aux2 = ones(alimentos,3)
aux2[1:end, 1:2] .= 4.
aux2[1:end, 3] .= 9.
macros = df[1:end, 2:4].*aux2

,Carb,Prot,Gord
,Float64,Float64,Float64
1,112.4,10.0,1.8
2,95.2,5.6,0.9
3,60.8,1.2,0.0
4,0.0,128.0,22.5
5,212.0,138.8,8.1
6,2.4,53.2,85.5
7,56.0,18.0,4.5
8,31.6,2.4,274.5
9,0.0,0.0,900.0


In [36]:
maxi = [macros[:, :Carb]'Limite_sup macros[:, :Prot]'Limite_sup macros[:, :Gord]'Limite_sup]
maxi .> [carb_cal prot_cal gord_cal]

1×3 BitArray{2}:
 1  1  1

# Modelo Matemático
***

In [209]:
optimizer = Juniper.Optimizer
nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)
model = Model(optimizer_with_attributes(optimizer, "nl_solver" => nl_solver))#,"mip_solver"=>mip_solver))

@variable(model, quantidade_alimento[i = 1:alimentos] ≥ 0, start = 2)
#@variable(model, quantidade[i = 1:alimentos], Bin)

@objective(model, Max, sum(quantidade_alimento[i] for i = 1:alimentos)/10) #/10 retorna a quantidade de alimentos em kg

#@constraint(model, meta_calorica,  quantidade_alimento[i]*sum(macros[i,1:3]) - basa ≤ 1e-2)

@constraint(model, limitante_comida[i = 1:alimentos],  quantidade_alimento[i] ≤ df[i,:Sup])

@constraint(model, meta_carb, -1e-4 ≤ sum(quantidade_alimento[i]*macros[i,1] for i = 1:alimentos) - carb_cal  ≤ 1e-4)

@constraint(model, meta_prot, -1e-4 ≤ sum(quantidade_alimento[i]*macros[i,2] for i = 1:alimentos) - prot_cal ≤ 1e-4)

@constraint(model, meta_gord, -1e-4 ≤ sum(quantidade_alimento[i]*macros[i,3] for i = 1:alimentos) - gord_cal ≤ 1e-4)

""

""

In [212]:
optimize!(model)

if typeof(objective_value(model)) == typeof(1.)
    g = value.(quantidade_alimento)
    #q = value.(quantidade)

    posic = findall(x -> x > 1e-5, g)

    d = DataFrame([df[posic,1] 100*g[posic]], [:Alimento, :Quantidade])
end
d

nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("print_level") => 0])
feasibility_pump  : false
log_levels        : [:Options, :Table, :Info]


┌ Info: no explicit NLP constraints or objective provided using @NLconstraint or @NLobjective macros
└ @ Juniper C:\Users\T-Gamer\.julia\packages\Juniper\gf4lD\src\MOI_wrapper\MOI_wrapper.jl:345


gain_mu           : 0.833

#Variables: 9
#IntBinVar: 0
#Constraints: 15
#Linear Constraints: 15
#Quadratic Constraints: 0
#NonLinear Constraints: 0
Obj Sense: Max

Incumbent using start values: 1.6321976591728062
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 0.03000020980834961
Relaxation Obj: 1.6321976591728062
Obj: 1.6321976591728062


,Alimento,Quantidade
,Any,Any
1,Arroz,500.0
2,Banana,250.0
3,Maçã,150.0
4,Frango,77.8099
5,Leite,145.391
6,Ovo,200.0
7,Feijão,200.0
8,Maionese,100.0
9,Azeite,8.99624


# Mais detalhes ao modelo
***

In [22]:
import XLSX
using JuMP, Juniper, Ipopt
using LinearAlgebra
using DataFrames

if ~occursin("Dieta", pwd())
    try
        cd("Desktop/Dieta")
    catch
        nothing
    end
end

In [2]:
#Utilizar com ativos fisicamente, objetivo: aumento de peso e massa magra.
#SUPERESTIMA O BASAL
function basal_Harris_Benedict(peso_kg, altura_cm, idade, sexo = 0)
    if sexo == 0 #homem
        return 66 + (13.8*peso_kg) + (5*altura_cm) - (6.8*idade)
    else #mulher
        return 665 + (9.6*peso_kg) + (1.9*altura_cm) - (4.7*idade)
    end
end

basal_Harris_Benedict (generic function with 2 methods)

In [3]:
function basal_FAO_OMS(peso_kg, idade, sexo = 0)
    if sexo == 0 #homem
        if 10 ≤ idade < 18
            return (17.686*peso_kg) + 658.2
        elseif  18 ≤ idade < 30
            return (15.057*peso_kg) + 692.2
        elseif 30 ≤ idade < 60
            return (11.472*peso_kg) + 873.1
        else
            return (11.711*peso_kg) + 587.7
        end
    else #mulher
        if 10 ≤ idade < 18
            return (13.384*peso_kg) + 692.6
        elseif  18 ≤ idade < 30
            return (14.818*peso_kg) + 486.6
        elseif 30 ≤ idade < 60
            return (8.126*peso_kg) + 845.6
        else
            return 	(9.082*peso_kg) + 658.5
        end
    end
end

basal_FAO_OMS (generic function with 2 methods)

In [4]:
#objetivo de emagrecimento.
#MAIS PRECISA QUE Harris_Benedict
function basal_Mifflin_St_Jeor(peso_kg, altura_cm, idade, sexo = 0)
    if sexo == 0 #homem
        return (10*peso_kg) + (6.25*altura_cm) - (5*idade) + 5
    else #mulher
        return (10*peso_kg) + (6.25*altura_cm) - (5*idade) - 161
    end
end

basal_Mifflin_St_Jeor (generic function with 2 methods)

In [6]:
#individuos ativos fisicamente e que possuem alto volume muscular e baixo percentual de gordura.
function basal_Cunningham_Tinsley(peso_kg, massa_magra; tipo = 1)
    if tipo == 1 #Cunningham (MLG)
        return (22*massa_magra) + 500
    elseif tipo == 2 #Tinsley (MLG)
        return (25.9*massa_magra) + 284
    else #Tinsley (P)
        return (24.8*peso_kg) + 10
    end
end

basal_Cunningham_Tinsley (generic function with 1 method)

In [7]:
kg = 64
age = 21
altura = 162
massa_magra = kg*(1 - (22.15 + 6)/100) #22.15% de gordura, 6% de óssos e etc.
atividades = "Ativo"

"Ativo"

# Verificando o Gasto Calórico Basal e Total
***

In [39]:
set_basal = [basal_Harris_Benedict(kg_m, altura_m, age_m),
            basal_FAO_OMS(kg_m, age_m),
            basal_Mifflin_St_Jeor(kg_m, altura_m, age_m),
            basal_Cunningham_Tinsley(kg_m, massa_magra_m, tipo = 1),
            basal_Cunningham_Tinsley(kg_m, massa_magra_m, tipo = 2),
            basal_Cunningham_Tinsley(kg_m, massa_magra_m, tipo = 3)]
println("Gasto Basal:")
name = ["Harris Benedict", "FAO OMS", "Mifflin St.Jeor", "Cunningham Tinsley_1", "Cunningham Tinsley_2", "Cunningham Tinsley_3"]
for i = 1:6
    println(name[i], " - ", set_basal[i])
end

Gasto Basal:
Harris Benedict - 1616.4
FAO OMS - 1655.848
Mifflin St.Jeor - 1552.5
Cunningham Tinsley_1 - 1511.6480000000001
Cunningham Tinsley_2 - 1474.9856
Cunningham Tinsley_3 - 1597.2


In [40]:
DataFrame(Nomes = name, Basal = set_basal)

,Nomes,Basal
,String,Float64
1,Harris Benedict,1616.4
2,FAO OMS,1655.85
3,Mifflin St.Jeor,1552.5
4,Cunningham Tinsley_1,1511.65
5,Cunningham Tinsley_2,1474.99
6,Cunningham Tinsley_3,1597.2


In [43]:
basa = (set_basal[1] + set_basal[3])/2
println("João - ", basa)

João - 1584.45


In [45]:
atividades = atividades_m
gasto_total = 1.
if atividades == "Sedentario"
    gasto_total = 1.2*basa #1 a 1.39
elseif atividades == "Pouco ativo"
    gasto_total = 1.45*basa #1.4 a 1.59
elseif atividades == "Ativo"
    gasto_total = 1.75*basa #1.6 a 1.89
else #Muito Ativo
    gasto_total = 2.2*basa #1.9 a 2.5
end
basa = gasto_total
println("Gasto Total Calórico - ", gasto_total)

Gasto Total Calórico - 2772.7875


# Definindo os Alimentos
***

In [40]:
xf = XLSX.readxlsx("tabela.xlsx")
tamanho_final = string(xf["Tabela1"])[40:end-1] #tamanho inteiro [35:end-1]  da tabela, "A1:FX203"
sh = xf[string("Tabela1!A1:",tamanho_final)] #pego a tabela hidr do A3 até o tamanho_final

elementos = [3; 179; 222; 410; 456; 488; 567; 524; 260]
#peso = [5 2.5 2.5 5 1.5 2.5 2.5 1.5 1.5]
elementos .+= 1

col = [2; 4; 5; 6; 7]

df = sh[elementos,col]
aux1 = replace.(df[1:end, 1:end-1], "Tr" => "0")
aux1 = replace.(aux1, "NA" => "0")
aux = parse.(Float64,replace.(aux1[1:end, 2:end], "," => "."))

df = [df[1:end, 1] aux df[1:end, end]]

9×5 Array{Any,2}:
 "Arroz"     28.1   2.5    0.2  128
 "Banana"    23.8   1.4    0.1   92
 "Maçã"      15.2   0.3    0.0   56
 "Frango"     0.0  32.0    2.5  159
 "Leite"     53.0  34.7    0.9  362
 "Ovo"        0.6  13.3    9.5  146
 "Feijão"    14.0   4.5    0.5   77
 "Maionese"   7.9   0.6   30.5  302
 "Azeite"     0.0   0.0  100.0  884

2372.7875

In [72]:
basalll = basa - 400
prot_cal, gord_cal = kg_m.*[2.0*4 0.5*9]
carb_cal = basalll - prot_cal - gord_cal

carb_g = carb_cal/4
prot_g = prot_cal/4
gord_g = gord_cal/9

alimentos = length(elementos)
aux2 = ones(alimentos,3)
aux2[1:end, 1:2] .= 4.
aux2[1:end, 3] .= 9.
macros = df[1:end, 2:4].*aux2

,Carb,Prot,Gord
,Float64,Float64,Float64
1,112.4,10.0,1.8
2,95.2,5.6,0.9
3,60.8,1.2,0.0
4,0.0,128.0,22.5
5,212.0,138.8,8.1
6,2.4,53.2,85.5
7,56.0,18.0,4.5
8,31.6,2.4,274.5
9,0.0,0.0,900.0


In [73]:
aux_ones = ones(3)
vet_cal = [carb_cal, prot_cal, gord_cal]
DataFrame(Macros = ["Carboidrato", "Proteina", "Gordura"], Calorias = vet_cal)

,Macros,Calorias
,String,Float64
1,Carboidrato,1572.79
2,Proteina,512.0
3,Gordura,288.0


In [74]:
Limite_inf, Limite_sup = ones(length(elementos),1),ones(length(elementos),1)

Limite_inf[:] = [100; 100; 100; 100; 50; 50; 50; 50; 10]/100

Limite_sup[:] = [500; 300; 200; 500; 500; 250; 250; 100; 100]/100

Limite_inf = convert(Array{Any,2}, Limite_inf)
Limite_sup = convert(Array{Any,2}, Limite_sup)

DataFrame([Int64.(Limite_inf*100) sh[elementos,[2]] Int64.(Limite_sup*100)], ["Inf g", "Alimento", "Sup g"])

,Inf g,Alimento,Sup g
,Any,Any,Any
1,100,Arroz,500
2,100,Banana,300
3,100,Maçã,200
4,100,Frango,500
5,50,Leite,500
6,50,Ovo,250
7,50,Feijão,250
8,50,Maionese,100
9,10,Azeite,100


In [75]:
maxi = [macros[:, :Carb]'Limite_sup macros[:, :Prot]'Limite_sup macros[:, :Gord]'Limite_sup]
maxi .> [carb_cal prot_cal gord_cal]

1×3 BitArray{2}:
 1  1  1

In [86]:
optimizer = Juniper.Optimizer
nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)
model = Model(optimizer_with_attributes(optimizer, "nl_solver" => nl_solver))#,"mip_solver"=>mip_solver))

@variable(model, quantidade_alimento[i = 1:alimentos])
@variable(model, quantidade[i = 1:alimentos], Bin)

#@objective(model, Max, sum(quantidade)/basa + sum(quantidade_alimento[i] for i = 1:alimentos)/10) #/10 retorna a quantidade de alimentos em kg
@objective(model, Max, sum(quantidade_alimento + quantidade)/alimentos)
#@constraint(model, meta_calorica,  quantidade_alimento[i]*sum(macros[i,1:3]) - basa ≤ 1e-2)

@NLconstraint(model, limitante_comida[i = 1:alimentos], quantidade_alimento[i] ≤ quantidade[i]*Limite_sup[i])

@constraint(model, limitante_comida_inf[i = 1:alimentos], quantidade_alimento[i] ≥ quantidade[i]*Limite_inf[i])

@constraint(model, meta_carb, -1e-4 ≤ sum(quantidade_alimento[i]*macros[i,1] for i = 1:alimentos) - carb_cal  ≤ 1e-4)

@constraint(model, meta_prot, -1e-4 ≤ sum(quantidade_alimento[i]*macros[i,2] for i = 1:alimentos) - prot_cal ≤ 1e-4)

@constraint(model, meta_gord, -1e-4 ≤ sum(quantidade_alimento[i]*macros[i,3] for i = 1:alimentos) - gord_cal ≤ 1e-4)

optimize!(model)

nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("print_level") => 0])
feasibility_pump  : false
log_levels        : [:Options, :Table, :Info]

#Variables: 18
#IntBinVar: 9
#Constraints: 24
#Linear Constraints: 15
#Quadratic Constraints: 0
#NonLinear Constraints: 9
Obj Sense: Max

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 0.05900001525878906
Relaxation Obj: 2.7238610890136763

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    0       2                2.62                        2.72             3.9%    0.1       1         -     

#branches: 1
Obj: 2.621525419357661


In [87]:
if typeof(objective_value(model)) == typeof(1.)
    g = value.(quantidade_alimento)
    q = value.(quantidade)

    posic = findall(x -> x > 1e-5, g)

    d = [df[posic,1] g[posic]]
end
println("Kg - ",sum(g/10))

Kg - 1.6593728704412842


In [101]:
g*100

9-element Array{Float64,1}:
 500.0000059939296
 300.0000039939978
 200.00000299365738
  -9.95929755851904e-7
 216.6059508180097
 182.61474651336573
 250.00000349173362
   9.944510661103518e-7
  10.152156638069586

In [100]:
Limite_sup*100

9×1 Array{Float64,2}:
 500.0
 300.0
 200.0
 500.0
 500.0
 250.0
 250.0
 100.0
 100.0

In [107]:
DataFrame(Alimento = d[1:end,1], Quantidade = d[1:end, 2]*100, sup = Limite_sup[posic], inf = Limite_inf[posic])

,Alimento,Quantidade,sup,inf
,Any,Float64,Any,Any
1,Arroz,500.0,5.0,1.0
2,Banana,300.0,3.0,1.0
3,Maçã,200.0,2.0,1.0
4,Leite,216.606,5.0,0.5
5,Ovo,182.615,2.5,0.5
6,Feijão,250.0,2.5,0.5
7,Azeite,10.1522,1.0,0.1


In [94]:
df[1:end,1]

9-element Array{Any,1}:
 "Arroz"
 "Banana"
 "Maçã"
 "Frango"
 "Leite"
 "Ovo"
 "Feijão"
 "Maionese"
 "Azeite"

In [109]:
Limite_inf

9×1 Array{Any,2}:
 1.0
 1.0
 1.0
 1.0
 0.5
 0.5
 0.5
 0.5
 0.1